## This is an example on how to leverage Picsellia SDK and tensorflow 2 to train a simple Image Classifier 

We suppose that you did not trained a first classifier before and want to start from scratch !

If you want to retrieve a trained model and start from it, please check [this notebook](https://google.com)

In [ ]:
from picsellia import Client 
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import models


### Pre-requisite 

🥑 In order to use this notebook, you need to have an [**Experiment created on Picsellia** and a **Classification Dataset attached**](https://google.com).

  - You can learn how to it [here](https://google.com)
  
  - You can find your API Token [here](https://app.picsellia.com/profile#token)


In [ ]:
PICSELLIA_API_TOKEN = "" 
project_name = ""
experiment_name = ""

#### 1 - Let's Initialize Picsellia Client and fetch your project and experiment

In [ ]:
client = Client(
    api_token=PICSELLIA_API_TOKEN,
)

project = client.get_project(
    name=project_name
)



#### 2 - Fetch your already created experiment in order to store and log everything to Picsellia 🥑

By passing `tree` and `with_files` to True, you will automatically get a folder architecture like:

- `experiment_name`/
    - checkpoint/
    - config/
    - exported_model/
    - images/
    - metrics/
    - records/
    - results/

In [ ]:
experiment = project.get_experiment(
    name=experiment_name,
    with_files=True,
    tree=True
)


#### 3 - Download your Dataset and create the train-test split repartition 📚

`train_test_split`  is recommended to be stored inside Picsellia, this way you will be able to access the validation interface and have full visibility over your training data 👊

If you want to know more about our train_test_split format, here is the [documentation page](https://google.com)


⚠️ You need to download annotations first to download the pictures.

In [ ]:
experiment.dl_annotation()
experiment.dl_picures()
experiment.train_test_split(prop=0.8)

train_split = {
    "x": experiment.categories,
    "y": experiment.train_repartition,
    "image_list": experiment.train_list_id
}
test_split = {
    'x': experiment.categories,
    'y': experiment.test_repartition,
    'image_list': experiment.eval_list_id
}

experiment.log(
    name="train-split",
    data=train_split,
    type="bar",
    replace=True
)

experiment.log(
    name="test-split",
    data=test_split,
    type="bar",
    replace=True
)


#### 4 - Create the Labelmap to and create the right folder tree

`labelmap` is needed for Picsellia to map your verbose labels *(i.e "cat", "dog", "hot-dog")* to your categorical labels *(i.e 1, 2, 3)*.

-> You can find more info about the labelmap format [here](https://google.com)

In [ ]:
labels_index = [e for e in range(1, len(experiment.categories)+1)]

labelmap = dict(zip(labels_index, experiment.categories))

experiment.log(
    name='labelmap', 
    data=labelmap, 
    type='labelmap', 
    replace=True
)

splits = ["train", "validation"]

for split in splits:
    if not split in os.listdir("images"):
        os.mkdir(os.path.join("images", split))
    for category in experiment.categories:
        dirpath = os.path.join("images", split)
        if not os.path.isdir(dirpath):
            os.mkdir(dirpath)

### 5 - Let's move your pictures to the right folders

In [ ]:
train_dir = "images/train"
validation_dir = "images/validation"

for image in experiment.dict_annotations["annotations"]:
    label = image["annotations"][0]["label"]
    image_id = image["internal_picture_id"]
    filename = image["external_picture_url"]
    if image_id in experiment.train_list_id:
        os.rename(os.path.join('images', filename), os.path.join(train_dir, label, filename))
    elif image_id in experiment.eval_list_id:
        os.rename(os.path.join('images', filename), os.path.join(validation_dir, label, filename))

### 6 - Let's get the parameters setup in Picsellia UI for your training

🥑 We are also going to create the Dataset from folder thanks to keras method `image_dataset_from_directory()`

In [ ]:
parameters = experiment.get_data("parameters")

train_dataset = image_dataset_from_directory(
    directory=train_dir,
    shuffle=True,
    batch_size=parameters["batch_size"],
    image_size=(parameters["image_size"], parameters["image_size"])
)

validation_dataset = image_dataset_from_directory(
    directory=validation_dir,
    shuffle=True,
    batch_size=parameters["batch_size"],
    image_size=(parameters["image_size"], parameters["image_size"])
)

### 7 - Let's visualize some Images and Labels to check everything is okk 😄

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(train_dataset.class_names[labels[i]])
    plt.axis("off")

#### 8 - Create your Test Dataset and make batches for training 

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

# Let's be really sure that it's alright :D

print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

#### 9 - Configure your Dataset TF config and perform data augmentation 🤔

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.title(train_dataset.class_names[labels[i]])
    plt.axis('off')

#### 10 - Preprocess your Images (rescaling) and define the PicselliaLogger Callback to see your training in real-time 🔬

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)
IMG_SHAPE = parameters["image_size"] + (3,)


class PicselliaLogger(Callback):
    def on_epoch_end(self, epoch, logs={}):
        for log_name, log_value in logs.items():
            experiment.log(log_name, [float(log_value)], 'line')

callback_list = [PicselliaLogger()]

#### 16 - Download the pre-trained model and Fine Tune the Top Layers of your models

In the feature extraction experiment, you were only training a few layers on top of an MobileNet V2 base model. The weights of the pre-trained network were not updated during training.

One way to increase performance even further is to train (or "fine-tune") the weights of the top layers of the pre-trained model alongside the training of the classifier you added. The training process will force the weights to be tuned from generic feature maps to features associated specifically with the dataset.

Also, you should try to fine-tune a small number of top layers rather than the whole MobileNet model. In most convolutional networks, the higher up a layer is, the more specialized it is. The first few layers learn very simple and generic features that generalize to almost all types of images. As you go higher up, the features are increasingly more specific to the dataset on which the model was trained. The goal of fine-tuning is to adapt these specialized features to work with the new dataset, rather than overwrite the generic learning.

In [ ]:
model = models.load_model(os.path.join(experiment.base_dir, "model.h5"))
history_fine = model.fit(
    x=train_dataset,
    epochs=parameters["fine_tune_epochs"],
    initial_epoch=0,
    callbacks=callback_list,
    validation_data=validation_dataset
)

#### 17 - Store your trained Model in Picsellia to use it later 🥑

In [ ]:
models_dir = os.path.join(experiment.base_dir, 'models')
tf.saved_model.save(model, os.path.join(models_dir, 'saved_model'))
model.save(os.path.join(models_dir, 'keras_model', 'model.h5'))
experiment.store("keras_model", os.path.join(models_dir, 'keras_model','model.h5'))
experiment.store("model-latest", os.path.join(models_dir, 'saved_model'), zip=True)

#### 18 - Perform Evaluation on your validation dataset and send it to Picsellia to visualize it interactively 🥑

In [ ]:
y_true = []
y_pred = []
for i in range(len(validation_dataset)):
    image_batch, label_batch = validation_dataset.as_numpy_iterator().next()
    predictions = model.predict_on_batch(image_batch).flatten()
    # Apply a sigmoid since our model returns logits
    predictions = tf.nn.sigmoid(predictions)
    predictions = tf.where(predictions < 0.5, 0, 1)
    y_true.extend(label_batch)
    y_pred.extend(predictions.numpy())

confusion_matrix = {
    'categories': experiment.categories,
    'values': confusion_matrix(y_true, y_pred).tolist()
}
experiment.log('confusion-matrix', confusion_matrix, 'heatmap')

# We set a parameters `fine_tune` to True to access it and directly perform fine-tuning without training the first layers next time
parameters["fine_tune"] = True
experiment.log(
    name="parameters",
    data=parameters,
    type="table",
    replace=False
)

experiment.update(status="success")

print(
    """You visualize all your results on Picsellia Reports page :\n
    -> https://app.picsellia.com/{}/experiment/{}/{}/overview
        """.format(client.organization.id, project.id, experiment.id)
)
